In [ ]:
from astropy.io import fits
import numpy as np
import sep

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Ellipse
import matplotlib.colors as colors

rcParams['figure.figsize'] = [10.,8.]

In [ ]:
#read image into 2d numpy array
data = fits.getdata('image.fits')

In [ ]:
def img_analysis(image,thresh, bitswap=False):
    if bitswap:
        data = image.byteswap(inplace=True).newbyteorder()
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()
plt.show()

## Background subtraction

In [ ]:
# measure a spatially varying background on the image
bkg = sep.Background(data)
bkg = sep.Background(data, bw=64, bh=64, fw=3, fh=3)

# subtract the background
data_sub = data - bkg
# get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()
# bkg_image = np.array(bkg) # equivalent to above
# show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.show()


# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()
# show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()

# subtract the background
data_sub = data - bkg

## Object detection

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)
# how many objects were detected
len(objects)

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    
plt.savefig('image')

In [ ]:
# available fields
objects.dtype.names

## Aperture photometry

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

In [ ]:
#read image into 2d numpy array
data = fits.getdata("f105w.fits")

In [ ]:
#show the image
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()
plt.show()

## Background subtraction

In [ ]:
data = data.byteswap(inplace=True).newbyteorder()

# measure a spatially varying background on the image
bkg = sep.Background(data)
bkg = sep.Background(data, bw=64, bh=64, fw=3, fh=3)
# get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()

In [ ]:
bkg_rms = bkg.rms()
data_sub = data - bkg
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()

## Object detection

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)
len(objects)

In [ ]:
#plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    
plt.savefig('image 2')

In [ ]:
# available fields
objects.dtype.names

## Aperture photometry

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

In [ ]:
image = fits.getdata("image.fits")
image = image.byteswap(inplace=True).newbyteorder()
image = image/np.max(image)

f105w = fits.getdata("f105w.fits")
f105w = f105w.byteswap(inplace=True).newbyteorder()
f105w = f105w/np.max(f105w)

f125w = fits.getdata("f125w.fits")
f125w = f125w.byteswap(inplace=True).newbyteorder()
f125w = f125w/np.max(f125w)

f160w = fits.getdata("f160w.fits")
f160w = f160w.byteswap(inplace=True).newbyteorder()
f160w = f160w/np.max(f160w)

In [ ]:
histogram = plt.hist(f105w.flatten(), bins='auto')
objects = img_analysis("f105w.fits", thresh=1.5)

In [ ]:
im = (np.abs(np.stack((f160w, f125w, f105w)).transpose()))**(0.1)

In [ ]:
plt.imshow(f105w, vmin = np.abs(np.mean(f105w)-np.std(f105w)), vmax=np.abs(np.mean(f105w)+np.std(f105w)))

In [ ]:
plt.imshow(im, vmin=np.mean(im)-np.std(im), vmax=np.mean(im)+np.std(im), interpolation = "nearest")
plt.show()